In [ ]:
from typing import List, Tuple
from helper.helper_functions import load_dataset, save_model
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif

In [ ]:
data = load_dataset('../data/assignment2_income_cleaned.xlsx')

In [ ]:
# Splitting the data into features (X) and target (y)
X = data.drop(columns=['income'])
y = data['income']

In [ ]:
# Calculate chi-squared statistics and p-values between each feature and the target class
chi2_stat, p_values = chi2(X, y)

# Create a DataFrame to display chi-squared statistics and p-values
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Chi-squared statistic': chi2_stat, 'P-value': p_values})

# Sort the DataFrame by chi-squared statistic in descending order
feature_importance_df = feature_importance_df.sort_values(by='Chi-squared statistic', ascending=False)

# Print feature importances
print("Feature importances (Chi-squared test):")
print(feature_importance_df)

In [ ]:
# Calculate mutual information between each feature and the target class
mutual_info = mutual_info_classif(X, y)

# Create a DataFrame to display feature importances
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': mutual_info})

# Sort the DataFrame by importance
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Print feature importances
print("Feature importances (Mutual Information):")
print(feature_importance_df)